In [28]:
import pandas as pd
train_df = pd.read_table('./dataset/ratings_train.txt')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [29]:
train_df.loc[4,:].document

'사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다'

In [30]:
train_df.label.value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [31]:
# 데이터 전처리
import re

# 숫자를 공백으로
train_df = pd.read_table('./dataset/ratings_train.txt')
test_df = pd.read_csv('./dataset/ratings_test.txt',sep='\t')
test_df = test_df.fillna(' ')
train_df = train_df.fillna(' ')
train_df['document'] = train_df['document'].apply(lambda x: re.sub(r'\d+',' ',x))
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r'\d+',' ',x))

train_df.drop('id',axis=1,inplace=True)
test_df.drop('id',axis=1,inplace=True)

In [32]:
from konlpy.tag import Okt

okt = Okt()
def okt_tokenizer(text):
    # 인자로 들어온 text를 형태소 단어로 토큰화하여 리스트객체로 반환 - morphs
    tokens_ko = okt.morphs(text)
    return tokens_ko


In [33]:
# 21분~22분 소요됨
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer ,ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

C:\Users\chonh\anaconda3\envs\cakd5\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [35]:
# [과제] lr로 모델링 및 하이퍼 파라미터 튜닝 후 테스트 데이터로 평가하세요.
from sklearn.metrics import accuracy_score

lr = LogisticRegression(random_state=0)
params = {'C' : [1,3.5,4.5,5.5,10]}
grid_clf = GridSearchCV(lr,param_grid=params, cv=3,scoring='accuracy',verbose=0)
grid_clf.fit(tfidf_matrix_train,train_df['label'])
best_clf = grid_clf.best_estimator_

tfidf_matrix_test = tfidf_vect.transform(test_df['document'])
preds = best_clf.predict(tfidf_matrix_test)
accuracy = accuracy_score(test_df['label'],preds)

print(f'최적 하이퍼 파라미터: {grid_clf.best_params_}\ntrain에서 최고 accuracy: {grid_clf.best_score_:.3f}')
print(f'테스트 데이터에서 accuracy: {accuracy:.3f}')

C:\Users\chonh\anaconda3\envs\cakd5\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\chonh\anaconda3\envs\cakd5\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

최적 하이퍼 파라미터: {'C': 3.5}
train에서 최고 accuracy: 0.859
테스트 데이터에서 accuracy: 0.862
